In [1]:
# Local
username="neo4j"
password="qwer1234"
url="bolt://localhost:7687"
database="worknet-jobs"

In [2]:
from neo4j import GraphDatabase

with  GraphDatabase.driver(uri=url, auth=(username, password)) as driver:
    driver.verify_connectivity()
    session = driver.session(database=database)

In [67]:
# csv_file_name = "pre_processed_jobs_worknet.csv"
# csv_file_path = f"file:///{csv_file_name}"
csv_file_path = "https://raw.githubusercontent.com/kofsitho87/RAG/main/Neo4j%20vector%20%EA%B2%80%EC%83%89%EC%9D%84%20%ED%86%B5%ED%95%B4%20%EC%9E%A1%EA%B2%80%EC%83%89%ED%95%98%EA%B8%B0/data/pre_processed_jobs_worknet.csv"

session.run(f"""
LOAD CSV WITH HEADERS FROM '{csv_file_path}' AS row
WITH row.JobTitle as title
RETURN title
LIMIT 3
""").to_df()

,title
0,단순 포장반직원 모집합니다
1,단순포장 및 락크작업
2,육류 가공 식당납품


In [14]:
import pandas as pd

df = pd.read_csv("data/pre_processed_jobs_worknet.csv")
df.keys()

Index(['JobTitle', 'JobVolunteerExperience', 'RequiredExperienceMonths',
       'PreferredExperienceMonths', 'JobVolunteerEducation', 'WeeklyWorkdays',
       'Workdays', 'JobPayment', 'MinMonthlyPayment', 'MaxMonthlyPayment',
       'JobContractType', 'JobBenefits', 'CompanyName', 'CompanyIndustry',
       'CompanySize', 'CompanyLocation'],
      dtype='object')

In [3]:
csv_file_path = "https://raw.githubusercontent.com/kofsitho87/RAG/main/Neo4j%20vector%20%EA%B2%80%EC%83%89%EC%9D%84%20%ED%86%B5%ED%95%B4%20%EC%9E%A1%EA%B2%80%EC%83%89%ED%95%98%EA%B8%B0/data/pre_processed_jobs_worknet.csv"

query = """
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/kofsitho87/RAG/main/Neo4j%20vector%20%EA%B2%80%EC%83%89%EC%9D%84%20%ED%86%B5%ED%95%B4%20%EC%9E%A1%EA%B2%80%EC%83%89%ED%95%98%EA%B8%B0/data/pre_processed_jobs_worknet.csv" AS row

WITH row
MERGE (j:Job {name: trim(row.JobTitle)})
    SET j.required_experience_months = COALESCE(toInteger(row.RequiredExperienceMonths), null)
    SET j.preferred_experience_months = COALESCE(toInteger(row.PreferredExperienceMonths), null)
    SET j.payment = trim(row.JobPayment)
    SET j.min_monthly_payment = COALESCE(toInteger(row.MinMonthlyPayment), null)
    SET j.max_monthly_payment = COALESCE(toInteger(row.MaxMonthlyPayment), null)
    
WITH row, j
WHERE row.Workdays IS NOT NULL
MERGE (wd:WeeklyWorkdays {value: toInteger(row.Workdays)})
MERGE (j)-[:HAS_WEEKLY_WORKDAYS]->(wd)

WITH row, j
WHERE row.JobContractType IS NOT NULL
MERGE (ct:ContractType {name: trim(row.JobContractType)})
MERGE (j)-[:USES_CONTRACT_TYPE]->(ct)

WITH row, j
WHERE row.JobVolunteerEducation IS NOT NULL
MERGE (ve:VolunteerEducation {name: trim(row.JobVolunteerEducation)})
MERGE (j)-[:REQUIRES_VOLUNTEER_EDUCATION]->(ve)

WITH row, j
WHERE row.JobBenefits IS NOT NULL
FOREACH (
    item in split(row.JobBenefits, ' ') | 
    MERGE (b:Benefit {name: trim(item)})
    MERGE (j)-[:PROVIDES_BENEFIT]->(b)
)

// Company
WITH row, j
WHERE row.CompanyName IS NOT NULL
MERGE (c: Company {name: trim(row.CompanyName)})
	SET c.industry = COALESCE(trim(row.CompanyIndustry), null)
	SET c.size = COALESCE(trim(row.CompanySize), null)
    SET c.address = COALESCE(trim(row.CompanyLocation), null)
MERGE (c)-[:HAS_JOB_OPENING]->(j)
"""

result = session.run(query)

#### Benefit 항목에서 "식사제공(1식)", "식사제공(2식)", "식사제공(3식)", "식사제공(중식비지급)"을 "식사제공"으로 Node를 통일화하고 "PROVIDES_BENEFIT" relationship의 값에 "1식", "2식"에 대한 값을 할당

통합 Benefit 노드 생성 및 기존 관계 재구성:먼저, "식사제공"이라는 새로운 Benefit 노드를 생성하고, 기존의 식사 제공 노드를 이 새 노드로 교체하면서 관계의 속성을 업데이트합니다.

쿼리 최적화:이 과정에서 기존 노드와 관계를 삭제하기 전에 새로운 관계를 만들고 속성을 잘 설정해야 합니다. 이 방식으로 데이터의 일관성을 유지할 수 있습니다.

In [76]:
session.run("""
// "식사제공" Benefit 노드 생성
MERGE (unified:Benefit {name: "식사제공"})

// "식사제공" Benefit 노드에 연결되어 있던 기존 관계와 노드를 처리
WITH unified
MATCH (j:Job)-[r:PROVIDES_BENEFIT]->(b:Benefit)
WHERE b.name IN ["식사제공(1식)", "식사제공(2식)", "식사제공(3식)", "식사제공(중식비지급)"]

WITH j, b, unified, r,
	CASE b.name
		WHEN "식사제공(1식)" THEN 1
		WHEN "식사제공(중식비지급)" THEN 1
		WHEN "식사제공(2식)" THEN 2
		WHEN "식사제공(3식)" THEN 3
	END as meals
MERGE (j)-[new_r:PROVIDES_BENEFIT {meals: meals}]->(unified)
DELETE r // 기존 관계 삭제
DETACH DELETE b // 기존 노드 삭제
""")

In [88]:
session.run("""
MATCH (b:Benefit {name: '식사제공'})<-[r:PROVIDES_BENEFIT]-(j:Job)
WHERE r.meals IN [1, 2, 3]
RETURN r.meals, count(j) AS count
ORDER BY r.meals
""").to_df()

,r.meals,count
0,1,449
1,2,143
2,3,85


In [4]:
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI

graph = Neo4jGraph(
    url=url, 
    username=username, 
    password=password,
    database=database,
)

In [5]:
graph.refresh_schema()
print(graph.schema)

Node properties are the following:
Job {name: STRING, payment: STRING, min_monthly_payment: INTEGER, max_monthly_payment: INTEGER, preferred_experience_months: INTEGER, required_experience_months: INTEGER},WeeklyWorkdays {value: INTEGER},ContractType {name: STRING},VolunteerEducation {name: STRING},Benefit {name: STRING},Company {name: STRING, size: STRING, industry: STRING, address: STRING}
Relationship properties are the following:

The relationships are the following:
(:Job)-[:PROVIDES_BENEFIT]->(:Benefit),(:Job)-[:HAS_WEEKLY_WORKDAYS]->(:WeeklyWorkdays),(:Job)-[:USES_CONTRACT_TYPE]->(:ContractType),(:Job)-[:REQUIRES_VOLUNTEER_EDUCATION]->(:VolunteerEducation),(:Company)-[:HAS_JOB_OPENING]->(:Job)


## CREATE FULLTEXT INDEX

In [ ]:
#job_name_fulltext

graph.query("""
CREATE FULLTEXT INDEX job_name_fulltext 
FOR (j:Job) ON EACH [j.name]
OPTIONS {
  indexConfig: {
    `fulltext.analyzer`: 'cjk',
    `fulltext.eventually_consistent`: true
  }
}
""")

In [11]:
#volunteer_education_fulltext

graph.query("""
CREATE FULLTEXT INDEX volunteer_education_fulltext 
FOR (n:VolunteerEducation) ON EACH [n.name]
OPTIONS {
  indexConfig: {
    `fulltext.analyzer`: 'cjk',
    `fulltext.eventually_consistent`: true
  }
}
""")

[]

In [ ]:
#benefit_fulltext

graph.query("""
CREATE FULLTEXT INDEX benefit_fulltext 
FOR (b:Benefit) ON EACH [b.name]
OPTIONS {
  indexConfig: {
    `fulltext.analyzer`: 'cjk',
    `fulltext.eventually_consistent`: true
  }
}
""")

In [8]:
#company_address_fulltext

graph.query("""
CREATE FULLTEXT INDEX company_address_fulltext 
FOR (c:Company) ON EACH [c.address]
OPTIONS {
  indexConfig: {
    `fulltext.analyzer`: 'cjk',
    `fulltext.eventually_consistent`: true
  }
}
""")

[]

In [9]:
#company_industry
graph.query("""
CREATE FULLTEXT INDEX company_industry_fulltext 
FOR (c:Company) ON EACH [c.industry]
OPTIONS {
  indexConfig: {
    `fulltext.analyzer`: 'cjk',
    `fulltext.eventually_consistent`: true
  }
}
""")

[]

In [10]:
#company_name
graph.query("""
CREATE FULLTEXT INDEX company_name_fulltext 
FOR (c:Company) ON EACH [c.name]
OPTIONS {
  indexConfig: {
    `fulltext.analyzer`: 'cjk',
    `fulltext.eventually_consistent`: true
  }
}
""")

[]

In [6]:
graph.query("""
CREATE VECTOR INDEX `job_benefit`
FOR (b: Benefit) ON (b.embedding)
OPTIONS {indexConfig: {
 `vector.dimensions`: 1536,
 `vector.similarity_function`: 'cosine'
}}
""")

[]

In [64]:
# graph.query("""
# MATCH(j:Job)
# MERGE (j2:Job2 {name: trim(j.name)})
# """)

graph.query("""
CREATE VECTOR INDEX `job_name2`
FOR (j: Job2) ON (j.embedding)
OPTIONS {indexConfig: {
 `vector.dimensions`: 1536,
 `vector.similarity_function`: 'cosine'
}}
""")

[]

In [37]:
graph.query("""
CREATE VECTOR INDEX `job_name`
FOR (j: Job) ON (j.embedding)
OPTIONS {indexConfig: {
 `vector.dimensions`: 1536,
 `vector.similarity_function`: 'cosine'
}}
""")

[]

In [52]:
graph.query("""
CREATE FULLTEXT INDEX `job_name_fulltext`
FOR (j:Job) ON EACH [j.name]
OPTIONS {
  indexConfig: {
    `fulltext.analyzer`: 'cjk',
    `fulltext.eventually_consistent`: true
  }
}
""")

[]

In [63]:
graph.query("""
CALL db.index.fulltext.queryNodes("job_name_fulltext", "식당보조") 
YIELD node, score
RETURN score, node.name as job
ORDER BY score DESC
LIMIT 10
""")

[{'score': 4.155252456665039, 'job': '정육식당 보조 구인(내/외국인)'},
 {'score': 2.9305131435394287, 'job': '육류 가공 식당납품'},
 {'score': 2.7983179092407227, 'job': '구내식당 찬모님 급구.'},
 {'score': 2.2833163738250732, 'job': '청담동순도리 식당 주방 / 홀서빙 직원 모집'},
 {'score': 2.2188167572021484, 'job': '용접 보조 조공 제관 보조 구인'},
 {'score': 2.126753807067871, 'job': '내국인, 외국인 구분 없이 채용. 식당 홀 서빙. 초보자 환영'},
 {'score': 2.00286865234375, 'job': '주방보조'},
 {'score': 1.990283489227295, 'job': '새마을운동중앙회 연수원식당 조리직원 채용'},
 {'score': 1.9386987686157227, 'job': '보조교사 구함 (보조인력)/외국인환영'},
 {'score': 1.903559923171997, 'job': '떡 제조 보조원'}]

In [15]:
from langchain_openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings()

In [34]:
search_embedding = embedding_model.embed_query("교통편을 제공하고 월급이 300만원이 넘는 일자리")

In [36]:
import pandas as pd

list_data = graph.query("""
CALL db.index.vector.queryNodes('job_benefit', 3, $embedding)
YIELD node AS benefit, score
                        
//MATCH (j:Job)-[:PROVIDES_BENEFIT]->(benefit)
            
RETURN benefit.name as benefit, apoc.number.format(score, '#.###') AS score
""", {
    "embedding": search_embedding
})

# list_data
result_df = pd.DataFrame(list_data)
result_df[:30]

,benefit,score
0,기숙사제공,0.914
1,통근버스제공,0.91
2,직원대출제도,0.903


In [61]:
from langchain_community.vectorstores import Neo4jVector
from langchain_community.vectorstores.neo4j_vector import SearchType
from langchain_openai import OpenAIEmbeddings

job_name_existing_graph = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    url=url,
    username=username,
    password=password,
    database=database,
    index_name="job_name",
    node_label="Job",
    text_node_properties=["name"],
    embedding_node_property="embedding",
    search_type=SearchType.HYBRID
)

In [43]:
import pandas as pd

df = pd.read_csv("data/pre_processed_jobs_worknet.csv")

In [46]:
df[df['JobTitle'].str.contains('식당')]['JobTitle']

2                              육류 가공 식당납품 
367                          구내식당 찬모님 급구. 
591              청담동순도리 식당 주방 / 홀서빙 직원 모집 
699                     정육식당 보조 구인(내/외국인) 
886    내국인, 외국인 구분 없이 채용. 식당 홀 서빙. 초보자 환영 
891                새마을운동중앙회 연수원식당 조리직원 채용 
Name: JobTitle, dtype: object

In [62]:
source_docs = job_name_existing_graph.similarity_search(
    query="식당보조",
    k=10, 
    # filter={"payment_type": "월급", "pay_amount": {"$gt": 3000000}}
)

source_docs

[Document(page_content='\nname: 용접 보조 조공 제관 보조 구인', metadata={'min_monthly_payment': 2200000, 'payment': '월급 220만원 ~ 240만원', 'max_monthly_payment': 2400000}),
 Document(page_content='\nname: 정육식당 보조 구인(내/외국인)', metadata={'min_monthly_payment': 2100000, 'payment': '월급 210만원 이상'}),
 Document(page_content='\nname: 떡 제조 보조원', metadata={'min_monthly_payment': 2200000, 'payment': '월급 220만원 ~ 250만원', 'max_monthly_payment': 2500000}),
 Document(page_content='\nname: 주방보조', metadata={'min_monthly_payment': 2600000, 'payment': '월급 260만원 이상'}),
 Document(page_content='\nname: 용접사 구합니다.', metadata={'required_experience_months': 120, 'min_monthly_payment': 3000000, 'payment': '월급 300만원 ~ 300만원', 'max_monthly_payment': 3000000}),
 Document(page_content='\nname: 사무관리 및 보조원 구합니다.', metadata={'min_monthly_payment': 2070000, 'payment': '월급 207만원 이상'}),
 Document(page_content='\nname: 주방보조 구합니다', metadata={'min_monthly_payment': 2700000, 'payment': '월급 270만원 ~ 310만원', 'max_monthly_payment': 3100000}),
 D

In [ ]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

store = Neo4jVector.from_existing_index(
    embedding=OpenAIEmbeddings(),
    url=url,
    username=username,
    password=password,
    database=database,
    # search_type="vector" # vector or hybrid
    index_name="job_benefit_index",
)

In [91]:
from langchain.chains import GraphCypherQAChain
llm = ChatOpenAI(model="gpt-4", temperature=0)

chain = GraphCypherQAChain.from_llm(
    graph=graph, 
    cypher_llm=llm,
    qa_llm=llm,
    validate_cypher=True,
    verbose=True,
)

In [92]:
chain.run("월급이 300만원이 넘고 서울에 있는 일자리가 있는지 찾아줘")

/Users/heewungsong/anaconda3/envs/visa_chatbot1/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (j:Job)-[:HAS_WEEKLY_WORKDAYS]->(ww:WeeklyWorkdays), (j)-[:USES_CONTRACT_TYPE]->(ct:ContractType), (j)-[:REQUIRES_VOLUNTEER_EDUCATION]->(ve:VolunteerEducation), (j)-[:PROVIDES_BENEFIT]->(b:Benefit), (c:Company)-[:HAS_JOB_OPENING]->(j) WHERE j.min_monthly_payment > 3000000 AND c.address = 'Seoul' RETURN j, ww, ct, ve, b, c
Full Context:
[]

> Finished chain.


'제가 알고 있는 정보가 없어서, 월급이 300만원이 넘고 서울에 있는 일자리에 대해 알려드릴 수 없습니다.'

In [93]:
chain.run("월급이 300만원 이상인 일자리 2개만 알려줘")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (j:Job) WHERE j.min_monthly_payment >= 3000000 RETURN j LIMIT 2
Full Context:
[{'j': {'max_monthly_payment': 3500000, 'name': '[이천시 가좌리] 신축빌라 공사현장 잡역부 모집', 'preferred_experience_months': 12, 'payment': '월급 350만원 ~ 350만원', 'min_monthly_payment': 3500000}}, {'j': {'max_monthly_payment': 3800000, 'name': '식육포장 처리 직원 모집', 'preferred_experience_months': 24, 'payment': '월급 300만원 ~ 380만원', 'min_monthly_payment': 3000000}}]

> Finished chain.


'[이천시 가좌리] 신축빌라 공사현장 잡역부 모집과 식육포장 처리 직원 모집이 월급이 300만원 이상인 일자리입니다.'